# Drake Davis
# Project Proposal Jupyter Notebook

In [ ]:
import time, re
from datetime import datetime, timedelta, date
from pathlib import Path
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# Kaggle data setup

##### Using: https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset?resource=download

In [ ]:
KAGGLE_CSV      = Path(r"movies_metadata.csv")
HEADLESS        = True                       
TARGET_STUDIOS  = ["Warner Bros.", "Universal", "Disney", "Sony", "Paramount"]
OUT_FILE        = "daily_boxoffice_2017_2020.csv"

In [29]:
# Setup

earliest_date = date(2017, 1, 1)
latest_date   = date(2020, 12, 31)


#   Build URLs
base_url = "https://www.the-numbers.com/box-office-chart/daily/"
daily_urls = [
    base_url + d.strftime("%Y/%m/%d")
    for d in pd.date_range(earliest_date, latest_date, freq="D")

]


# Scraping weekend charts from the-numbers.com

In [ ]:
# Web scraping script 1

opts = Options()
if HEADLESS:
    opts.add_argument("--headless=new")
opts.add_argument("--log-level=3")
driver = webdriver.Chrome(options=opts)

rows_out = []
for url in daily_urls:
    driver.get(url)
    time.sleep(1.0)                            # polite wait

    page_date = "/".join(url.split("/")[-3:])  # YYYY/MM/DD
    for tr in driver.find_elements(By.CSS_SELECTOR, "table tbody tr"):
        tds = tr.find_elements(By.TAG_NAME, "td")
        if len(tds) < 11:
            continue

        try:
            studio = tds[3].text.strip()
            if not any(s.lower() in studio.lower() for s in TARGET_STUDIOS):
                continue

            title  = tds[2].text.strip()
            daily  = float(re.sub(r"[$,]", "", tds[4].text) or 0)
            theaters_txt = tds[7].text.replace(",", "")
            theaters = int(theaters_txt) if theaters_txt.isdigit() else None
            total  = float(re.sub(r"[$,]", "", tds[9].text) or 0)

            rows_out.append({
                "date": page_date,
                "title": title,
                "studio": studio,
                "daily_gross": daily,
                "theaters": theaters,
                "total_gross": total
            })
        except Exception as e:
            print(f"[{page_date}] Skip row — {e}")

driver.quit()

In [ ]:
# Save to csv/xlsx

df_daily = pd.DataFrame(rows_out)
print(f"Scraped {len(df_daily):,} rows across {df_daily['date'].nunique():,} days.")
df_daily.to_csv(OUT_FILE, index=False)

Scraped 21,196 rows across 1,314 days.
✅  Saved daily_boxoffice_2017_2020.csv


# Integration/Cleaning

In [37]:
KAGGLE_CSV = Path("movies_metadata.csv")
NUMBERS_CSV = Path("daily_boxoffice_2017_2020.csv")          
OUT_FILE    = "movies_boxoffice_2017_2020_merged.csv"

In [38]:
def clean_title(txt: str) -> str:
    """Upper‑case alphanumerics only (for deterministic joins)."""
    return re.sub(r"[^A-Z0-9 ]", "", str(txt).upper()).strip()

In [39]:
# Loading kaggle data

meta = (
    pd.read_csv(KAGGLE_CSV, low_memory=False,
                usecols=["title", "budget", "runtime", "genres", "release_date"])
      .assign(
          release_date=lambda d: pd.to_datetime(d["release_date"], errors="coerce"),
          budget=lambda d: pd.to_numeric(d["budget"], errors="coerce"),
          runtime=lambda d: pd.to_numeric(d["runtime"], errors="coerce"),
      )
      .dropna(subset=["title", "release_date"])
      .query("release_date.dt.year.between(2017, 2020)")
)

meta["release_year"] = meta["release_date"].dt.year
meta["title_clean"]  = meta["title"].apply(clean_title)


In [40]:
# Loading the numbers data

box = (
    pd.read_csv(NUMBERS_CSV, parse_dates=["date"])
      .rename(columns={"date": "calendar_date"})
)

# keep the first calendar appearance as “release” row
box = (
    box.sort_values(["title", "calendar_date"])
        .groupby("title", as_index=False)
        .first()
)

box["release_year"] = box["calendar_date"].dt.year
box["title_clean"]  = box["title"].apply(clean_title)


In [41]:
# Merge

merged = (
    box.merge(meta, on=["title_clean", "release_year"], how="left",
              suffixes=("", "_kaggle"))
)

print(f"Rows after merge: {len(merged):,}")
print(f"Matched Kaggle budgets for {merged['budget'].notna().sum():,} titles "
      f"({merged['budget'].notna().mean():.1%})")

merged.to_csv(OUT_FILE, index=False)
print(f"  Saved {OUT_FILE}")

Rows after merge: 303
Matched Kaggle budgets for 35 titles (11.6%)
  Saved movies_boxoffice_2017_2020_merged.csv
